## Comparing a set of models

First, I'll compare a set of out-of-the-box regression models to select the best performing model for hyperparameter tuning.

The target variable - rented_bike_count - is skewed, but I don't just want to transform it using yeo-johnson. I want to transform it, train the model, then transform it back, to make sure that I'm testing the actual predicted value, not the *transformed* predicted value.

Surprisingly, I immediately beat the ~91% R2 I've seen other people post online, suggesting that either my feature engineering and/or preprocessing is better - or that the introduction of the pollution data made a significant difference.

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import warnings
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.compose import TransformedTargetRegressor
from sklearn.inspection import permutation_importance
from joblib import dump

In [2]:
# Use if working in colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Use if working in colab
# set working directory to project directory
%cd '/content/drive/My Drive/github/bicycles-seoul'

/content/drive/My Drive/github/bicycles-seoul


In [4]:
df = pd.read_csv('data/processed/02-processed-for-training.csv')

In [5]:
X = df.drop('rented_bike_count', axis=1)
y = df['rented_bike_count']

# First split: create training set (70%) and a temporary set (30%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42
)

# Second split: split the temporary set equally into validation (15%) and test (15%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,
    random_state=42
)

print(f"Train set: {X_train.shape}, {y_train.shape}")
print(f"Validation set: {X_val.shape}, {y_val.shape}")
print(f"Test set: {X_test.shape}, {y_test.shape}")

Train set: (5924, 88), (5924,)
Validation set: (1270, 88), (1270,)
Test set: (1270, 88), (1270,)


In [21]:
# This function introduces yeo-johnson transformation for the target variable,
# but transforms the target back before validation.

def train_and_compare_models(X_train, y_train, X_val, y_val):
    # Convert Series to NumPy array, then reshape
    y_train_array = y_train.values.reshape(-1, 1)
    y_val_array = y_val.values.reshape(-1, 1)

    pt = PowerTransformer(method='yeo-johnson')
    y_train_trans = pt.fit_transform(y_train_array).ravel()

    models = {
        "LinearRegression": LinearRegression(),
        "Lasso": Lasso(),
        "Ridge": Ridge(),
        "KNeighborsRegressor": KNeighborsRegressor(),
        "SVR": SVR(),
        "DecisionTree": DecisionTreeRegressor(),
        "RandomForest": RandomForestRegressor(),
        "ExtraTreeRegressor": ExtraTreeRegressor(),
        "GradientBoostingRegressor": GradientBoostingRegressor(),
        "XGBRegressor": XGBRegressor(use_label_encoder=False, eval_metric='rmse'),
        "LightGBM": LGBMRegressor(),
        "MLPRegressor": MLPRegressor()
    }

    results = []

    for name, model in models.items():
        model.fit(X_train, y_train_trans)

        # Predictions in transformed space
        y_pred_trans = model.predict(X_val)

        # Invert predictions to original scale
        y_pred = pt.inverse_transform(y_pred_trans.reshape(-1, 1)).ravel()

        # Evaluate on the original scale
        r2 = r2_score(y_val_array, y_pred)
        rmse = np.sqrt(mean_squared_error(y_val_array, y_pred))

        results.append({
            "Model": name,
            "R2": r2,
            "RMSE": rmse
        })

    results_df = pd.DataFrame(results).sort_values(by="R2", ascending=False)
    return results_df


In [22]:
results_df = train_and_compare_models(X_train, y_train, X_val, y_val)
print(results_df)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000751 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1472
[LightGBM] [Info] Number of data points in the train set: 5924, number of used features: 88
[LightGBM] [Info] Start training from score -0.000000
                        Model        R2        RMSE
4                         SVR  0.940787  155.242931
10                   LightGBM  0.937039  160.080895
11               MLPRegressor  0.932177  166.146974
9                XGBRegressor  0.931673  166.762638
6                RandomForest  0.908149  193.350645
8   GradientBoostingRegressor  0.830820  262.407908
0            LinearRegression  0.813849  275.255308
2                       Ridge  0.813760  275.320836
7          ExtraTreeRegressor  0.797638  286.9

## Which features are most important for SVR?

Here I use the permutation importance method to find out which features are most important for the performance of SVR.

I want to know whether the high R2 scores I'm seeing here are because of the way I've handled the existing data, or whether they're down to the introduction of the pollution data.

The answer seems to be that my approach to the pre-processing of the usual data is behind the high performance of a non-finetuned SVR: the most important features are the seasons, temperature, and - most of all - the binary 'raining' variable I created.

The pollution data is in the top 40 variables, but it doesn't rank particularly high.

In [36]:
############################################
# 1) Wrap SVR with a TransformedTargetRegressor
############################################
pt = PowerTransformer(method='yeo-johnson')
svr = SVR(kernel='rbf', C=1.0)
model = TransformedTargetRegressor(regressor=svr, transformer=pt)

# 2) Fit the model (Yeo-Johnson transform is applied to y_train internally)
model.fit(X_train, y_train)

# 3) Evaluate on the original y scale (automatically inversely transformed)
y_pred = model.predict(X_val)
r2 = r2_score(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)

print(f"R² on original scale:  {r2:.3f}")
print(f"RMSE on original scale: {rmse:.3f}")

############################################
# 4) Permutation importance
############################################
# This also works with the original y because the TransformedTargetRegressor
# internally handles the target transform for training/prediction.
result = permutation_importance(model, X_val, y_val, n_repeats=5, random_state=42)
importances = result.importances_mean

# 5) Pair with your feature names
feature_importance = pd.DataFrame({
    'feature': X_val.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print(feature_importance)


R² on original scale:  0.941
RMSE on original scale: 155.243
               feature  importance
25           Seasons_3    0.258831
74      Temperature(C)    0.137659
17             Hour_18    0.082278
3               Hour_4    0.065623
86             raining    0.065144
..                 ...         ...
80                PM10    0.001478
79  log(Snowfall (cm))    0.001275
44              day_19    0.000981
38              day_13    0.000805
40              day_15    0.000307

[88 rows x 2 columns]


In [37]:
feature_importance.head(40)

,feature,importance
25,Seasons_3,0.258831
74,Temperature(C),0.137659
17,Hour_18,0.082278
3,Hour_4,0.065623
86,raining,0.065144
73,week_day_6,0.043340
18,Hour_19,0.041884
2,Hour_3,0.041834
4,Hour_5,0.041317
85,any_radiation,0.040420


## SVM hyperparamter tuning

Interestingly, SVM tuning doesn't result in a significant increase in performance compared to the out-of-the-box implementation. Perhaps I should try tuning the next most successful technique - LightGBM?

In [38]:
# 1) Define the SVR model wrapped with Yeo-Johnson transformation
svr = TransformedTargetRegressor(
    regressor=SVR(),
    transformer=PowerTransformer(method='yeo-johnson')
)

# 2) Set up a grid of hyperparameters
# Note the 'regressor__' prefix to access SVR's parameters within TransformedTargetRegressor
param_grid = {
    'regressor__C': [0.1, 1, 10],
    'regressor__gamma': ['scale', 'auto'],
    'regressor__kernel': ['rbf', 'linear']
}

# 3) Define the GridSearchCV
grid_search = GridSearchCV(
    estimator=svr,
    param_grid=param_grid,
    scoring='r2',         # to maximize R²
    cv=5,                 # 5-fold cross-validation
    n_jobs=-1,            # use all CPU cores
    verbose=1             # print out progress
)

# 4) Fit the grid search on your training data
grid_search.fit(X_train, y_train)

# 5) Get the best model and best hyperparameters
best_svr = grid_search.best_estimator_
print("Best Params:", grid_search.best_params_)

# 6) Evaluate on the test set
y_pred = best_svr.predict(X_test)

# Calculate evaluation metrics
try:
    # For scikit-learn >= 0.22
    rmse = mean_squared_error(y_test, y_pred, squared=False)
except TypeError:
    # For older versions
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

r2 = r2_score(y_test, y_pred)

print(f"Test R²: {r2:.3f}")
print(f"Test RMSE: {rmse:.3f}")


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Params: {'regressor__C': 10, 'regressor__gamma': 'scale', 'regressor__kernel': 'rbf'}
Test R²: 0.938
Test RMSE: 160.917


In [ ]:
dump(best_svr, "bicycles_seoul/models/best_svr_model.joblib")

## LightGBM hyperparameter tuning

By taking the next best-performing technique (LightGBM) and tuning the parameters using GridsearchCV, I can achieve 0.944 R2 on the test data.

We'll take this to be the best performing model.

In [6]:
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import PowerTransformer
from lightgbm import LGBMRegressor
from joblib import dump

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# 1) Define the LightGBM model wrapped with Yeo-Johnson transformation
lgbm = TransformedTargetRegressor(
    regressor=LGBMRegressor(random_state=42),
    transformer=PowerTransformer(method='yeo-johnson')
)

# 2) Set up a grid of hyperparameters for LightGBM
# Note the 'regressor__' prefix to access LGBMRegressor's parameters within TransformedTargetRegressor
param_grid = {
    'regressor__n_estimators': [100, 200],
    'regressor__learning_rate': [0.01, 0.1, 0.2],
    'regressor__num_leaves': [31, 50, 100],
    'regressor__max_depth': [-1, 10, 20],
    'regressor__min_child_samples': [20, 30, 40],
    'regressor__subsample': [0.6, 0.8, 1.0],
    'regressor__colsample_bytree': [0.6, 0.8, 1.0]
}

# 3) Define the GridSearchCV
grid_search = GridSearchCV(
    estimator=lgbm,
    param_grid=param_grid,
    scoring='r2',         # maximize R²
    cv=5,                 # 5-fold cross-validation
    n_jobs=-1,            # use all CPU cores
    verbose=1             # print out progress
)

# 4) Fit the grid search on your training data
grid_search.fit(X_train, y_train)

# 5) Get the best model and best hyperparameters
best_lgbm = grid_search.best_estimator_
print("Best Params:", grid_search.best_params_)

# 6) Evaluate on the test set
y_pred = best_lgbm.predict(X_test)

# Calculate evaluation metrics
try:
    # For scikit-learn >= 0.22
    rmse = mean_squared_error(y_test, y_pred, squared=False)
except TypeError:
    # For older versions
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

r2 = r2_score(y_test, y_pred)

print(f"Test R²: {r2:.3f}")
print(f"Test RMSE: {rmse:.3f}")

Fitting 5 folds for each of 1458 candidates, totalling 7290 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000657 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1472
[LightGBM] [Info] Number of data points in the train set: 5924, number of used features: 88
[LightGBM] [Info] Start training from score -0.000000
Best Params: {'regressor__colsample_bytree': 1.0, 'regressor__learning_rate': 0.1, 'regressor__max_depth': 20, 'regressor__min_child_samples': 20, 'regressor__n_estimators': 200, 'regressor__num_leaves': 50, 'regressor__subsample': 0.6}
Test R²: 0.944
Test RMSE: 152.994


In [ ]:
# Best Params: {'regressor__colsample_bytree': 1.0, 'regressor__learning_rate': 0.1, 'regressor__max_depth': 20, 'regressor__min_child_samples': 20, 'regressor__n_estimators': 200, 'regressor__num_leaves': 50, 'regressor__subsample': 0.6}

In [6]:
# LGBM with optimal parameters
lgbm = TransformedTargetRegressor(
    regressor=LGBMRegressor(
        colsample_bytree=1.0,
        learning_rate=0.1,
        max_depth=20,
        min_child_samples=20,
        n_estimators=200,
        num_leaves=50,
        subsample=0.6,
        random_state=42  # Ensures reproducibility
    ),
    transformer=PowerTransformer(method='yeo-johnson')
)

# fit the model
lgbm.fit(X_train, y_train)

# predict on test
y_pred = lgbm.predict(X_test)

# evaluation metrics
try:
    # For scikit-learn >= 0.22
    rmse = mean_squared_error(y_test, y_pred, squared=False)
except TypeError:
    # For older versions
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

r2 = r2_score(y_test, y_pred)

print(f"Test R²: {r2:.3f}")
print(f"Test RMSE: {rmse:.3f}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000914 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1472
[LightGBM] [Info] Number of data points in the train set: 5924, number of used features: 88
[LightGBM] [Info] Start training from score -0.000000
Test R²: 0.944
Test RMSE: 152.994


/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The LGBMRegressor or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


In [9]:
dump(lgbm, "bicycles_seoul/models/best_lgbm.joblib")

['bicycles_seoul/models/best_lgbm.joblib']